In [ ]:
include("../path.jl")
include("../ToolBox/ToolBox.jl")
using .ToolBox
using .Geometry

## Chose the burst

In [ ]:
burst_n = 4;

In [ ]:
# load meta and pod
master_meta = Load.slc_meta(master_meta_path);
master_precise_orbit = Load.precise_orbit(master_pod_path,master_meta["t_0"]);
slave_meta = Load.slc_meta(slave_meta_path);
slave_precise_orbit = Load.precise_orbit(slave_pod_path,slave_meta["t_0"]);

meta = (master_meta,slave_meta)
precise_orbit = (master_precise_orbit,slave_precise_orbit);

In [ ]:
lines_per_burst = master_meta["lines_per_burst"];

In [ ]:
master_view = [3000:6500,3000:9000]
master_data = Load.slc_data(master_data_path,master_view)
SlcUtil.show_img(master_data,0.995)

In [ ]:
# load DEM
footprint = SlcUtil.footprint(master_meta, master_view)
latlon_window = ((minimum(footprint[1]),maximum(footprint[1])),(minimum(footprint[2]),maximum(footprint[2])))
dem = Load.dem(dem_path, latlon_window; nan_fill= 0, padding=[90,90]);

## Handle ovelap

In [ ]:
first_line_mosaic_master = master_meta["burst_meta"]["first_line_mosaic"][burst_n]
first_line_mosaic_slave = slave_meta["burst_meta"]["first_line_mosaic"][burst_n]
over_lap_master = lines_per_burst*(burst_n-1)+1 - (first_line_mosaic_master)
over_lap_slave = lines_per_burst*(burst_n-1)+1 - (first_line_mosaic_slave)

In [ ]:
mosaic_view = SlcUtil.mosaic_view(master_meta,master_view)

In [ ]:
lut = look_up_table(mosaic_view,meta,precise_orbit,dem,stride=(2,8));

In [ ]:
lut["master_line"] = lut["master_line"] .+ over_lap_master
lut["slave_line"] = lut["slave_line"] .+ over_lap_slave;

In [ ]:
slave_line = Misc.interp_grid(lut["master_line"] ,lut["master_sample"],
    reshape(lut["slave_line"],(length(lut["master_line"]),length(lut["master_sample"])))
    ,master_view[1], master_view[2])
slave_line = reshape(slave_line,:)

slave_sample = Misc.interp_grid(lut["master_line"] ,lut["master_sample"],
    reshape(lut["slave_sample"],(length(lut["master_line"]),length(lut["master_sample"])))
    ,master_view[1], master_view[2]);

slave_sample= reshape(slave_sample,:);

### resample

In [ ]:
dim = (length(master_view[1]),length(master_view[2]))

In [ ]:
padd = 2

slave_view = round(Int,minimum(slave_line )-padd): round(Int,maximum(slave_line )+padd),
            round(Int,minimum(slave_sample)-padd): round(Int,maximum(slave_sample)+padd)
slave_data = Load.slc_data(slave_data_path, slave_view);

In [ ]:
v_mid = Geometry.mid_burst_speed(precise_orbit[2], meta[2]);

In [ ]:
phi = SlcUtil.phase_ramp(Misc.flatten(slave_view...)..., burst_n, meta[2], v_mid[burst_n]);
de_ramp = slave_data .* reshape(exp.(-phi .* im),size(slave_data));

In [ ]:
slave_resample = Misc.resample(slave_view,de_ramp,slave_line,slave_sample)
slave_resample = reshape(slave_resample, dim);

In [ ]:
phi = SlcUtil.phase_ramp(slave_line, slave_sample, burst_n, meta[2], v_mid[burst_n]) 
reramp = slave_resample .* reshape(exp.(phi .* im),dim);


In [ ]:
infero = master_data .* conj.(reramp);
SlcUtil.plot_phase(infero)

In [ ]:
master_line, master_sample = Misc.flatten(master_view...)

c = 299792458
range_pixel_spacing =  c/(2*meta[1]["range_sampling_rate"])
lambda =  c/meta[1]["radar_frequency"]
lut["slave_sample"]
flat = exp.(4*pi.*(master_sample.-slave_sample).*range_pixel_spacing./lambda.*im)
flat = reshape(flat,dim);


In [ ]:
SlcUtil.plot_phase(infero.*flat)

In [ ]:
SlcUtil._phase_colorbar()